In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model


In [1]:
import tensorflow as tf
print(tf.__version__)  # TensorFlow version
print(tf.keras.__version__)  # Keras version (if available)


2.16.1
3.3.3


In [2]:
# Path to your dataset
data_dir = '/kaggle/input/garbage-classification-v2/garbage-dataset'  # Replace with the path to your data folder


In [3]:
# Image data generator for training and validation with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Use 20% of the data for validation
)

# Create train and validation generators
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),  # Image size compatible with InceptionV3
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 15530 images belonging to 10 classes.
Found 3877 images belonging to 10 classes.


In [4]:
# Load InceptionV3 as the base model without the top classification layer
inception_base = tf.keras.applications.InceptionV3(include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the InceptionV3 base to keep pre-trained weights
inception_base.trainable = False

# Define the custom model
model = Sequential([
    Input(shape=(224, 224, 3)),  # Input layer with the required shape
    inception_base,              # Add the InceptionV3 base model
    Flatten(),                   # Flatten the output from the base model
    
    # Add dense layers as specified
    Dense(128, activation="relu"),
    Dense(128, activation="relu"),
    Dense(128, activation="relu"),
    Dense(128, activation="relu"),
    Dense(128, activation="relu"),
    
    # Final output layer with softmax activation for classification
    Dense(train_generator.num_classes, activation="softmax")
])

# Compile the model with Adam optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Plot the model architecture
plot_model(model, show_shapes=True, to_file="model_structure.png")

# Print the model summary
print(model.summary())


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inception_v3 (Functional)       │ (None, 5, 5, 2048)     │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 51200)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     6,553,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,423,850 (108.43 MB)

 Trainable params: 6,621,066 (25.26 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

None


In [5]:
# Train the model using the train and validation data
history = model.fit(
    train_generator,
    epochs=10,  # Adjust the number of epochs based on your needs
    validation_data=validation_generator
)


Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1724930732.636996     145 service.cc:145] XLA service 0x7c8d200166b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724930732.637054     145 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1724930732.637058     145 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


  2/486 ━━━━━━━━━━━━━━━━━━━━ 46s 97ms/step - accuracy: 0.0391 - loss: 3.0867   

I0000 00:00:1724930743.621079     145 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


486/486 ━━━━━━━━━━━━━━━━━━━━ 374s 721ms/step - accuracy: 0.7443 - loss: 0.8949 - val_accuracy: 0.8852 - val_loss: 0.3765
Epoch 2/10
486/486 ━━━━━━━━━━━━━━━━━━━━ 252s 512ms/step - accuracy: 0.8997 - loss: 0.3179 - val_accuracy: 0.8881 - val_loss: 0.3507
Epoch 3/10
486/486 ━━━━━━━━━━━━━━━━━━━━ 250s 508ms/step - accuracy: 0.9217 - loss: 0.2537 - val_accuracy: 0.9012 - val_loss: 0.3158
Epoch 4/10
486/486 ━━━━━━━━━━━━━━━━━━━━ 247s 501ms/step - accuracy: 0.9371 - loss: 0.1982 - val_accuracy: 0.9017 - val_loss: 0.3513
Epoch 5/10
486/486 ━━━━━━━━━━━━━━━━━━━━ 248s 504ms/step - accuracy: 0.9406 - loss: 0.1872 - val_accuracy: 0.8955 - val_loss: 0.3445
Epoch 6/10
486/486 ━━━━━━━━━━━━━━━━━━━━ 247s 502ms/step - accuracy: 0.9483 - loss: 0.1739 - val_accuracy: 0.9097 - val_loss: 0.3070
Epoch 7/10
486/486 ━━━━━━━━━━━━━━━━━━━━ 245s 499ms/step - accuracy: 0.9514 - loss: 0.1486 - val_accuracy: 0.9108 - val_loss: 0.3228
Epoch 8/10
486/486 ━━━━━━━━━━━━━━━━━━━━ 254s 518ms/step - accuracy: 0.9552 - loss: 0.14

In [6]:
val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

122/122 ━━━━━━━━━━━━━━━━━━━━ 51s 416ms/step - accuracy: 0.9107 - loss: 0.3059
Validation Loss: 0.3103, Validation Accuracy: 0.9131


In [7]:
model.save('waste_classifier_inceptionv3_custom.h5')